In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import torch

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, mean,expr, avg, stddev
from pyspark.sql.functions import lag, coalesce, lit
from pyspark.sql.functions import to_date, date_format, to_timestamp
from pyspark.sql.window import Window
from pyspark.sql.functions import col, unix_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import warnings 
warnings.filterwarnings('ignore')

In [16]:
print(torch.__version__)  # Kiểm tra phiên bản PyTorch
print(torch.cuda.is_available())  # Kiểm tra GPU đã hoạt động chưa
print(torch.cuda.get_device_name(0))  # In tên GPU của bạn

2.5.1+cu121
True
NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [17]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Streaming Fraud Detection System") \
    .getOrCreate()    

In [18]:
train_path = r'D:/Data Science/Big Data Technology/Project/Streaming-Fraud-Detection/Streaming-Fraud-Detection/data/raw/fraudTrain.csv'
test_path = r'D:/Data Science/Big Data Technology/Project/Streaming-Fraud-Detection/Streaming-Fraud-Detection/data/raw/fraudTest.csv'   

In [19]:
train_data = spark.read.csv(train_path, header=True, inferSchema=True)
test_data = spark.read.csv(test_path, header=True, inferSchema=True)

In [20]:
df = train_data.union(test_data)
df.show(5)

+---+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|_c0|trans_date_trans_time|          cc_num|            merchant|     category|   amt|    first|   last|gender|              street|          city|state|  zip|    lat|     long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat| merch_long|is_fraud|
+---+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|  0|  2019-01-01 00:00:18|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97| Jennifer|  Banks|     F|      561 Perry 

In [21]:
df.printSchema()
print(f"Rows: {df.count()}, Columns: {len(df.columns)}")

root
 |-- _c0: integer (nullable = true)
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)

Rows: 1852394, Columns: 23


In [22]:
df.describe().show()

+-------+------------------+--------------------+-------------------+-------------+------------------+-------+-------+-------+--------------------+-------+-------+------------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-----------------+------------------+--------------------+
|summary|               _c0|              cc_num|           merchant|     category|               amt|  first|   last| gender|              street|   city|  state|               zip|              lat|              long|         city_pop|               job|           trans_num|           unix_time|        merch_lat|        merch_long|            is_fraud|
+-------+------------------+--------------------+-------------------+-------------+------------------+-------+-------+-------+--------------------+-------+-------+------------------+-----------------+------------------+-----------------+------------------+--------------------+---------

In [23]:
missing_values = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
missing_values.show()
df = df.dropna()

+---+---------------------+------+--------+--------+---+-----+----+------+------+----+-----+---+---+----+--------+---+---+---------+---------+---------+----------+--------+
|_c0|trans_date_trans_time|cc_num|merchant|category|amt|first|last|gender|street|city|state|zip|lat|long|city_pop|job|dob|trans_num|unix_time|merch_lat|merch_long|is_fraud|
+---+---------------------+------+--------+--------+---+-----+----+------+------+----+-----+---+---+----+--------+---+---+---------+---------+---------+----------+--------+
|  0|                    0|     0|       0|       0|  0|    0|   0|     0|     0|   0|    0|  0|  0|   0|       0|  0|  0|        0|        0|        0|         0|       0|
+---+---------------------+------+--------+--------+---+-----+----+------+------+----+-----+---+---+----+--------+---+---+---------+---------+---------+----------+--------+



In [24]:
df = df.withColumn("trans_date_trans_time", to_timestamp(col("trans_date_trans_time"), "dd/MM/yyyy HH:mm:ss"))
df = df.orderBy("trans_date_trans_time")
df.show(5)

+---+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|_c0|trans_date_trans_time|          cc_num|            merchant|     category|   amt|    first|   last|gender|              street|          city|state|  zip|    lat|     long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat| merch_long|is_fraud|
+---+---------------------+----------------+--------------------+-------------+------+---------+-------+------+--------------------+--------------+-----+-----+-------+---------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|  0|  2019-01-01 00:00:18|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97| Jennifer|  Banks|     F|      561 Perry 

In [25]:
columns_to_drop = ['first', 'last', 'street', 'city', 'state', 'zip', 'trans_num']
df = df.drop(*columns_to_drop)
df.show(5)

+---+---------------------+----------------+--------------------+-------------+------+------+-------+---------+--------+--------------------+----------+----------+------------------+-----------+--------+
|_c0|trans_date_trans_time|          cc_num|            merchant|     category|   amt|gender|    lat|     long|city_pop|                 job|       dob| unix_time|         merch_lat| merch_long|is_fraud|
+---+---------------------+----------------+--------------------+-------------+------+------+-------+---------+--------+--------------------+----------+----------+------------------+-----------+--------+
|  0|  2019-01-01 00:00:18|2703186189652095|fraud_Rippin, Kub...|     misc_net|  4.97|     F|36.0788| -81.1781|    3495|Psychologist, cou...|1988-03-09|1325376018|         36.011293| -82.048315|       0|
|  1|  2019-01-01 00:00:44|    630423337322|fraud_Heller, Gut...|  grocery_pos|107.23|     F|48.8878|-118.2105|     149|Special education...|1978-06-21|1325376044|49.159046999999994|-1

## Encoder categorical (StringIndexer)

In [26]:
from pyspark.ml.feature import StringIndexer

categorical_cols = ['merchant', 'category', 'gender', 'job']

for col_name in categorical_cols:
    indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_Indexer")
    df = indexer.fit(df).transform(df)
    
df = df.drop(*categorical_cols)
    
df.show(5)

+---+---------------------+----------------+------+-------+---------+--------+----------+----------+------------------+-----------+--------+----------------+----------------+--------------+-----------+
|_c0|trans_date_trans_time|          cc_num|   amt|    lat|     long|city_pop|       dob| unix_time|         merch_lat| merch_long|is_fraud|merchant_Indexer|category_Indexer|gender_Indexer|job_Indexer|
+---+---------------------+----------------+------+-------+---------+--------+----------+----------+------------------+-----------+--------+----------------+----------------+--------------+-----------+
|  0|  2019-01-01 00:00:18|2703186189652095|  4.97|36.0788| -81.1781|    3495|1988-03-09|1325376018|         36.011293| -82.048315|       0|           584.0|            11.0|           0.0|      129.0|
|  1|  2019-01-01 00:00:44|    630423337322|107.23|48.8878|-118.2105|     149|1978-06-21|1325376044|49.159046999999994|-118.186462|       0|           104.0|             1.0|           0.0|   

#### Spliting Into Training and Testing Datasets

In [27]:
split_index = int(df.count()* 0.8)
train_df  = df.limit(split_index)
test_df = df.subtract(train_df)

# Display row counts of the resulting DataFrames to verify the split
print(f"Training set row count: {train_df.count()}")
print(f"Testing set row count: {test_df.count()}")

Training set row count: 1481915
Testing set row count: 370479


In [30]:
feature_columns = [ "merchant_Indexer", "category_Indexer", "amt", "gender_Indexer", "lat", "long", "city_pop", "job_Indexer", "unix_time", "merch_lat", "merch_long", "age", "hour", "day", "month"]

In [31]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import unix_timestamp

# Convert the timestamp to a Unix timestamp (if not already).
#df = df.withColumn("unix_time", unix_timestamp("trans_date_trans_time").cast("int"))

# Create a data processing pipeline.
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
pipeline = Pipeline(stages=[assembler, scaler])

# Train the pipeline on the dataset.
transformer = pipeline.fit(df)

# Transform the data.
df = transformer.transform(df).select("trans_date_trans_time", "scaled_features", "is_fraud")
train_set = transformer.transform(train_df).select("trans_date_trans_time", "scaled_features", "is_fraud")
test_set = transformer.transform(test_df).select("trans_date_trans_time", "scaled_features", "is_fraud")

train_set.show(5)


IllegalArgumentException: age does not exist. Available: _c0, trans_date_trans_time, cc_num, amt, lat, long, city_pop, dob, unix_time, merch_lat, merch_long, is_fraud, merchant_Indexer, category_Indexer, gender_Indexer, job_Indexer